## Deliverable 2. Create a Customer Travel Destinations Map.

In [ ]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
sys.path.append("..")
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [ ]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df

In [ ]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))

max_temp = float(input("What is the maximum temperature you would like for your trip?"))

In [ ]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_data_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) &
                    (city_data_df["Max Temp"]>= min_temp)]

preferred_city_data_df

In [ ]:
# 4a. Determine if there are any empty rows.
preferred_city_data_df.count()

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_city_data_df.dropna()

clean_df

In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
# "location" : city,
"radius": 5000,
"type": "lodging",
"key": gkey }

#use iterrows

for index, row in hotel_df.iterrows():
    
    #get lat and lng from df 
    lat = row["Lat"]
    lng = row["Lng"]

    city = row["City"]

    #add location to params
    params["location"] = f"{lat},{lng}" 

    #url and api request

    print(f"Retrieving Results for Index{index}: {city} .")

    response = requests.get(base_url, params=params).json()

    #extract
    results = response["results"]

    try:
        print(f"{results[0]['name']} in {city}.")

        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    
    except:
        print("Could not find result...skipping.")

    print()

In [ ]:
#error: could not find hotel for any country - what is error in code? (same error as previous activity)
#  #set up base url 
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# # 6a. Set parameters to search for hotels with 5000 meters.
# params = {
#     "radius": 5000,
#     "type": "lodging",
#     "key": gkey
# }

# # 6b. Iterate through the hotel DataFrame.
# for index, row in hotel_df.iterrows():
#     # 6c. Get latitude and longitude from DataFrame
#     lat = row["Lat"]
#     lng = row["Lng"]

#     city = row["City"]
    
#     print(f"Retrieving Results for Index{index}: {city} .")

#     # 6d. Set up the base URL for the Google Directions API to get JSON data.
#     response = requests.get(base_url, params=params).json()

#     # 6e. Make request and retrieve the JSON data from the search. 
    
#     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
#     #extract
#     results = response["results"]

#     try:
#         print(f"{results[0]['name']} in {city}.")

#         hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    
#     except:
#         print("Could not find result...skipping.")

#     print()
        

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "Hotel DataFrame"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Description} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

#figure variable (plus center and zoom)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level =1.5)

#add heatlayer w variables (plus list comprehension, intensity, point radius)
heatmap_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heatmap_layer)

#marker layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig 